In [ ]:
import cv2
import os
from datetime import datetime

def detect_and_crop_license_plate_from_webcam():
    # Load the pre-trained Haar Cascade classifier for license plates
    plate_cascade = cv2.CascadeClassifier('haarcascade_plate_number.xml')

    # Start capturing video from the webcam
    cap = cv2.VideoCapture(0)

    # Create a directory to store the cropped images if it doesn't exist
    if not os.path.exists('cropped_images'):
        os.makedirs('cropped_images')

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        if not ret:
            break

        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect license plates in the frame
        plates = plate_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Draw rectangles around detected license plates
        for (x, y, w, h) in plates:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            # Crop the detected license plate region
            plate_roi = frame[y:y+h, x:x+w]

            # Generate a unique filename using current timestamp
            timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
            filename = f'cropped_images/cropped_plate_{timestamp}.jpg'

            # Save the cropped license plate image
            cv2.imwrite(filename, plate_roi)
            print(f"Saved {filename}")

        # Display the frame with license plate detection
        cv2.imshow('License Plate Detection', frame)

        # Press 'q' to exit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close all windows
    cap.release()
    cv2.destroyAllWindows()

# Call the function to start license plate detection, cropping, and saving from webcam
detect_and_crop_license_plate_from_webcam()

In [11]:
import pytesseract
from PIL import Image
import os
import re

# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def clean_text(text):
    # Remove any non-alphanumeric characters (keep only letters and numbers)
    return re.sub(r'[^a-zA-Z0-9]', '', text)

def process_image(image_location):
    try:
        # Open image file
        img = Image.open(image_location)
        
        # Use pytesseract to do OCR on the image
        text = pytesseract.image_to_string(img)

        cleaned_text = clean_text(text)
        print(f"{cleaned_text}\n")
        
        # Process the text to find specific patterns
        final_description = ''
        if len(cleaned_text) == 10 and \
           cleaned_text[:2].isalpha() and \
           cleaned_text[-4:].isdigit():
            final_description = cleaned_text

        return final_description if final_description else None
    except Exception as e:
        print(f"Error processing image {image_location}: {e}")
        return None

# Process all images in the cropped_images directory
cropped_images_dir = "cropped_images"
unique_texts = []

for filename in os.listdir(cropped_images_dir):
    if filename.endswith(".jpg"):
        img_loc = os.path.join(cropped_images_dir, filename)
        final_text = process_image(img_loc)
        if final_text and final_text not in unique_texts:
            unique_texts.append(final_text)

# Print the list of unique final_text values
for text in unique_texts:
    print(text)


MH20DV2366

MH20DV2366

WI20092366

MH20DV2366

MH20D2366

MH20D2366



MH20DV2366
WI20092366


In [12]:
suspected_plate = input("Enter the suspected plate: ")

Enter the suspected plate:  MH20DV2366


In [13]:
# Initialize a list to store potential matches
potential_matches = []

# Iterate through the unique texts list
for text in unique_texts:
  # Check for exact match
    if text == suspected_plate:
        # If exact match, add it to the list and break the loop
        potential_matches.append(text)
        break;
        
if len(potential_matches) == 0:
    # Check for approximate match (7 out of 10 characters similar)
    for match in unique_texts:
        count = 0
        for i in range(10):
            if suspected_plate[i] == match[i]:
                count += 1
        if (count >= 7):
            potential_matches.append(match)

# Print the potential matches
print("Potential Matches:", potential_matches)

Potential Matches: ['MH20DV2366']


In [34]:
import cv2
import os
from datetime import datetime
from PIL import Image
import pytesseract
import requests
import re

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def get_crime():
    url = 'https://p-wagon-backend.vercel.app/api/fetchCrimes'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            crime = response.json()
            return crime
        else:
            print('Error:', response.status_code)
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def extract_valid_licenses(crime_data):
    valid_licenses = []
    for key, value in crime_data.items():
        if value['status']:
            valid_licenses.append(value['details']['license'])
    
    return valid_licenses

def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9]', '', text)

def process_image(image_location):
    try:
        img = Image.open(image_location)
        
        text = pytesseract.image_to_string(img)

        cleaned_text = clean_text(text)
        
        final_description = ''
        if len(cleaned_text) == 10 and \
           cleaned_text[:2].isalpha() and \
           cleaned_text[-4:].isdigit():
            final_description = cleaned_text

        return final_description if final_description else None
    except Exception as e:
        print(f"Error processing image {image_location}: {e}")
        return None

def detect_and_crop_license_plate_from_webcam():
    plate_cascade = cv2.CascadeClassifier('haarcascade_plate_number.xml')

    cap = cv2.VideoCapture(0)

    if not os.path.exists('cropped_images'):
        os.makedirs('cropped_images')

    unique_texts = []

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        plates = plate_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in plates:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            plate_roi = frame[y:y+h, x:x+w]

            timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
            filename = f'cropped_images/cropped_plate_{timestamp}.jpg'

            cv2.imwrite(filename, plate_roi)
            
            final_text = process_image(filename)
            if final_text and final_text not in unique_texts:
                unique_texts.append(final_text)

        cv2.imshow('License Plate Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    return unique_texts

def find_potential_matches(unique_texts, suspected_plate):
    potential_matches = []

    if suspected_plate in unique_texts:
        potential_matches.append(suspected_plate)


    if len(potential_matches) == 0:
        for text in unique_texts:
            count = sum(1 for a, b in zip(suspected_plate, text) if a == b)
            if count >= 7:
                potential_matches.append(text)

    return potential_matches

def main():
    crime_data = get_crime()

    if crime_data:
        valid_licenses = extract_valid_licenses(crime_data)
        print("Valid Licenses with status True:")
        for license_plate in valid_licenses:
            print(license_plate)
    else:
        print("No crime data found.")
        
    unique_texts = detect_and_crop_license_plate_from_webcam()

    potential_matches = find_potential_matches(unique_texts, suspected_plate)

    print("Potential Matches:", potential_matches)

if __name__ == "__main__":
    main()

Valid Licenses with status True:
MH20DV2366
Potential Matches: ['MH20DV2366']


In [19]:
# MH13BN8454